In [ ]:
library(ggplot2)
library(dplyr)
library(lme4)
library(lmerTest)
library(xtable)
library(glmmTMB)
library(cowplot)
library(grid)

In [ ]:
franck_df <- read.csv('items.csv.reformat')
franck_df <- tidyr::separate(franck_df, condition, c("subj", "inter", "local"), sep=1:3)
franck_df$sample <- 100 - franck_df$sample
franck_df$max <- 100 - franck_df$max

franck_df$inter_match <- franck_df$subj == franck_df$inter
franck_df$local_match <- franck_df$subj == franck_df$local
franck_df$type = substr(franck_df$model, 5, 5)


franck_lm_df = subset(franck_df, substr(franck_df$model, 5,5) == "l")
franck_joint_df = subset(franck_df, substr(franck_df$model, 5,5) == "a")
summary(franck_df)

In [ ]:
means_lm <- (franck_lm_df %>% group_by(model, inter_match, local_match) 
                          %>% summarize(sample = mean(sample), max = mean(max)))
means_joint <- (franck_joint_df %>% group_by(model, inter_match, local_match) 
                                %>% summarize(sample = mean(sample), max = mean(max)))

means_inter <- (franck_df %>% group_by(type, inter_match) 
                          %>% summarize(sample = mean(sample), max = mean(max)))
means_local <- (franck_df %>% group_by(type, local_match) 
                          %>% summarize(sample = mean(sample), max = mean(max)))

In [ ]:
franck_human <- data.frame(subj=rep(c("S","P"), each=4),
                           inter=rep(c("S", "S", "P", "P"), times=2),
                           local=rep(c("S", "P"), times=4),
                           err=c(1.3, 0, 9.8, 4.5, 5.8, 9.4, 3.1, 1.3)/100)

franck_human$inter_match <- franck_human$inter == franck_human$subj
franck_human$local_match <- franck_human$local == franck_human$subj


franck_human$type <- "h"
human_inter <- franck_human %>% group_by(type, inter_match) %>% summarize(sample=mean(err * 100), max=0)
means_inter <- rbind(means_inter, human_inter)
means_inter

human_local <- franck_human %>% group_by(type, local_match) %>% summarize(sample=mean(err * 100), max=0)
means_local <- rbind(means_local, human_local)
means_local

In [ ]:
library(repr)
options(repr.plot.width=7, repr.plot.height=4)

x_order = c("l", "a", "h")

pallete3 = c("#318480", "#b4464f", "#c15733")

plt_1 <- ggplot(data=franck_df, aes(x=factor(type, level=x_order), y=sample/100)) +
#geom_bar(stat="summary", fun = mean, fill="#566e4b", width=0.6) + 
geom_point(data=means_inter, shape=16, size=5, aes(color=type)) + 
geom_line(data=means_inter, aes(group=type,color=type), size=2) +
labs(x="Syntactic Dist. Effect", 
     fill="", y="% Attraction Errors", title="") +
scale_y_continuous(labels=scales::percent_format(accuracy=2L)) +
scale_color_manual(labels=c("LM+CCG", 
                        "LM Only",
                        "Humans"), values=pallete3) +
scale_x_discrete(labels=c("l"="LM Only",
                          "a"="LM+CCG",
                          "h"="Humans")) +
coord_cartesian(ylim = c(0,0.5)) +
theme_bw() +
theme(axis.text.x=element_text(size=19, angle=45, hjust=1), axis.text.y=element_text(size=20),
      axis.title.y=element_text(size=20), axis.title.x=element_text(size=17),
      strip.text=element_text(size=20), legend.text=element_text(size=20), legend.position="none")

plt_1

plt_2 <- ggplot(data=franck_df, aes(x=factor(type, level=x_order), y=sample/100)) +
#geom_bar(stat="summary", fun = mean, fill="#566e4b", width=0.6) + 
geom_point(data=means_local, shape=16, size=5, aes(color=type)) + 
geom_line(data=means_local, aes(group=type, color=type), size=2) +
labs(x="Linear Dist. Effect", 
     fill="", y="% Attraction Errors", title="") +
scale_y_continuous(labels=scales::percent_format(accuracy=0), position="right") +
scale_color_manual(labels=c("LM+CCG", 
                        "LM Only",
                        "Humans"), values=pallete3) +
scale_x_discrete(labels=c("l"="LM Only",
                          "a"="LM+CCG",
                          "h"="Humans")) +
coord_cartesian(ylim = c(0,0.5)) +
theme_bw() +
theme(axis.text.x=element_text(size=19, angle=45, hjust=1), axis.text.y=element_blank(), 
      axis.title.y=element_blank(), axis.title.x=element_text(size=17),
      strip.text=element_text(size=20), legend.text=element_text(size=20), legend.position="none")

plt_2


cowplot::plot_grid(plt_1, plt_2, rel_widths=c(1.35,1))
ggsave("Franck_ccg_eff.png", width = 7, height = 4)